# Nueva prueba

# Reset the authentication token

In [ ]:
# 🔄 FULL RESET + RE-AUTHENTICATION WITH DRIVE PERMISSIONS
# 1️⃣ Remove any previous credentials that might be cached
home = os.path.expanduser("~")
ee_token_paths = [
    os.path.join(home, ".config", "earthengine", "credentials"),
    os.path.join(home, ".config", "earthengine", "private-key.json"),
    os.path.join(home, ".earthengine", "credentials")
]
for path in ee_token_paths:
    if os.path.exists(path):
        try:
            os.remove(path)
            print(f"🧹 Deleted old token: {path}")
        except Exception as e:
            print(f"⚠️ Couldn't remove {path}: {e}")

# 2️⃣ Reset any in-memory session (for notebooks)
try:
    ee.Reset()
except Exception:
    pass

In [16]:
# ============================================
# 🇦🇷 NDWI WATER POLYGONS — ARGENTINA IN TILES
# Exports multiple GeoJSONs to Drive and can merge them
# ============================================

# --- USER CONFIG ---
NDWI_THRESHOLD = 0.2               # NDWI water threshold
MIN_POLYGON_AREA = 300             # m²
CLOUD_COVER_MAX = 10               # %
SENTINEL_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'

# Smoothing (per polygon: close small internal holes without merging lakes)
SMOOTH_POLYGONS = True
SMOOTH_BUFFER_SIZE = 40            # meters (buffer + unbuffer)

# Tiling over Argentina (degrees)
TILE_SIZE_DEG = 1.5                # tile width/height (deg)
TILE_OVERLAP_DEG = 0.05            # small overlap to avoid edge artifacts

# Drive export
EXPORT_FOLDER = 'Azolla Filiculoides - Geojson Lagunas'
EXPORT_PREFIX = 'agua_ndwi'        # filename prefix

# Export/task control
MAX_ACTIVE_TASKS = 3               # GEE usually runs up to 2–3 concurrently
CHECK_EVERY_SEC = 30               # polling interval for task slots
REDUCE_SCALE = 10                  # meters (S2 native)
MAX_PIXELS = 1e10                  # be generous but safe

# Optional merge (LOCAL Python step after exports finish)
DO_LOCAL_MERGE = False             # set True to do an on-the-fly merge (requires files present locally or in Colab FS)
MERGE_GLOB = None                  # e.g. "/content/drive/MyDrive/Azolla Filiculoides - Geojson Lagunas/*.geojson"
OUT_MERGED = "merged_argentina.geojson"


# --- IMPORTS ---
import ee, time, math, os, glob, json
from pathlib import Path

# --- EE INIT ---
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# --- GEOMETRY: ARGENTINA ---
# Using US State Dept. simple borders (stable + public)
ARG = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Argentina')) \
    .geometry()

# --- TILING HELPERS ---
def get_bounds_deg(geom: ee.Geometry):
    # returns (minLon, minLat, maxLon, maxLat) in degrees (client-side)
    b = geom.bounds().coordinates().get(0).getInfo()  # [[minLon,minLat], [maxLon,minLat], [maxLon,maxLat], [minLon,maxLat], [minLon,minLat]]
    xs = [p[0] for p in b]
    ys = [p[1] for p in b]
    return min(xs), min(ys), max(xs), max(ys)

def generate_tiles_over_geometry(geom: ee.Geometry, size_deg: float, overlap_deg: float):
    """Client-side: builds a list of ee.Geometry rectangles covering 'geom'."""
    minx, miny, maxx, maxy = get_bounds_deg(geom)
    step = size_deg - overlap_deg
    tiles = []
    yi = miny
    while yi < maxy:
        xi = minx
        y_top = min(yi + size_deg, maxy)
        while xi < maxx:
            x_right = min(xi + size_deg, maxx)
            rect = ee.Geometry.Rectangle([xi, yi, x_right, y_top], proj='EPSG:4326', geodesic=False)
            # Intersect with ARG to avoid empty ocean tiles
            tile_geom = rect.intersection(geom, 1)
            # Check area > 0 (server-side, but we can trust intersection; small getInfo once is fine)
            try:
                area = tile_geom.area(1).getInfo()
            except Exception:
                area = 0
            if area and area > 0:
                tiles.append((xi, yi, tile_geom))
            xi += step
        yi += step
    return tiles

# --- PROCESSING PIPELINE ---
def latest_s2_low_cloud(geom: ee.Geometry):
    col = (ee.ImageCollection(SENTINEL_COLLECTION)
           .filterBounds(geom)
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER_MAX))
           .sort('system:time_start', False))
    # Fallback: if empty, relax cloud cover
    img = ee.Image(ee.Algorithms.If(col.size().gt(0), col.first(),
                                    ee.ImageCollection(SENTINEL_COLLECTION)
                                      .filterBounds(geom)
                                      .sort('system:time_start', False)
                                      .first()))
    return ee.Image(img)

def water_fc_for_geom(geom: ee.Geometry, ndwi_threshold: float):
    img = latest_s2_low_cloud(geom)
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    water = ndwi.gt(ndwi_threshold)
    water_int = water.updateMask(water).multiply(1).toInt()

    vec = water_int.reduceToVectors(
        geometry=geom,
        scale=REDUCE_SCALE,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='water',
        bestEffort=True,
        maxPixels=MAX_PIXELS
    )

    # add area and filter
    vec = vec.map(lambda f: f.set('area', f.geometry().area(1)))
    vec = vec.filter(ee.Filter.gt('area', MIN_POLYGON_AREA))

    # optional smoothing
    if SMOOTH_POLYGONS:
        def smooth(f):
            g = f.geometry().buffer(SMOOTH_BUFFER_SIZE).buffer(-SMOOTH_BUFFER_SIZE)
            return ee.Feature(g).copyProperties(f)
        vec = vec.map(smooth)

    return vec

# --- EXPORT HELPERS ---
def task_slots_available(max_concurrent=MAX_ACTIVE_TASKS):
    tasks = ee.batch.Task.list()
    active = sum(t.state in ('READY', 'RUNNING') for t in tasks)
    return max_concurrent - active

def wait_for_free_slot():
    while task_slots_available() <= 0:
        print(f"⏳ Waiting for a free export slot... ({CHECK_EVERY_SEC}s)")
        time.sleep(CHECK_EVERY_SEC)

def export_tile_fc_to_drive(fc: ee.FeatureCollection, name: str):
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        description=name,
        folder=EXPORT_FOLDER,
        fileNamePrefix=name,
        fileFormat='GeoJSON'
    )
    task.start()
    print(f"🚀 Started export: {name}.geojson  →  /{EXPORT_FOLDER}")
    return task

# --- MAIN: CREATE TILE EXPORTS ---
tiles = generate_tiles_over_geometry(ARG, TILE_SIZE_DEG, TILE_OVERLAP_DEG)
print(f"🧱 Tiles to process: {len(tiles)} (size={TILE_SIZE_DEG}°, overlap={TILE_OVERLAP_DEG}°)")
started = 0

for idx, (x, y, g) in enumerate(tiles, 1):
    # Build a deterministic tile label
    name = f"{EXPORT_PREFIX}_{int(NDWI_THRESHOLD*100)}_tile_{idx:03d}_lon{round(x,2)}_lat{round(y,2)}"
    try:
        fc = water_fc_for_geom(g, NDWI_THRESHOLD)
        # If collection empty, skip (cheap check via size().getInfo())
        if fc.size().getInfo() == 0:
            print(f"⚠️  Tile {idx:03d} has no water polygons. Skipping.")
            continue
        wait_for_free_slot()
        export_tile_fc_to_drive(fc, name)
        started += 1
    except Exception as e:
        print(f"❌ Error on tile {idx:03d}: {e}")

print(f"✅ Queued {started} export task(s). Drive folder: /{EXPORT_FOLDER}")
print("💡 Leave this cell running or re-run a small watcher cell to monitor ee.batch.Task.list() status.")


# --- OPTIONAL: LOCAL MERGE (after all exports complete) ---
# Set DO_LOCAL_MERGE=True and MERGE_GLOB to the folder pattern where the .geojsons are (local path or Colab FS).
if DO_LOCAL_MERGE:
    if MERGE_GLOB is None:
        # Try a common Colab Drive path guess:
        guess = f"/content/drive/MyDrive/{EXPORT_FOLDER}/*.geojson"
        print(f"MERGE_GLOB not set. Guessing: {guess}")
        MERGE_GLOB = guess

    files = sorted(glob.glob(MERGE_GLOB))
    if not files:
        print(f"⚠️ No files matched: {MERGE_GLOB}")
    else:
        all_feats = []
        for fp in files:
            try:
                with open(fp, "r", encoding="utf-8") as f:
                    data = json.load(f)
                if data.get("type") == "FeatureCollection":
                    all_feats.extend(data.get("features", []))
                elif data.get("type") == "Feature":
                    all_feats.append(data)
                else:
                    # assume geometry-only
                    all_feats.append({"type":"Feature","properties":{},"geometry":data})
            except Exception as e:
                print(f"⚠️ Skipping {fp}: {e}")

        merged = {"type":"FeatureCollection","features":all_feats}
        Path(OUT_MERGED).write_text(json.dumps(merged, ensure_ascii=False, indent=2), encoding="utf-8")
        print(f"🧩 Merged {len(files)} files → {OUT_MERGED}")

        # (Optional) dissolve overlapping polygons if Shapely is available:
        #   pip install shapely fastgeojson
        # and then convert to a single dissolved MultiPolygon. Keeping it simple here to avoid extra deps.


🧱 Tiles to process: 176 (size=1.5°, overlap=0.05°)
🚀 Started export: agua_ndwi_20_tile_001_lon-69.21_lat-55.05.geojson  →  /Azolla Filiculoides - Geojson Lagunas
🚀 Started export: agua_ndwi_20_tile_002_lon-67.76_lat-55.05.geojson  →  /Azolla Filiculoides - Geojson Lagunas
🚀 Started export: agua_ndwi_20_tile_003_lon-66.31_lat-55.05.geojson  →  /Azolla Filiculoides - Geojson Lagunas
⏳ Waiting for a free export slot... (30s)
⏳ Waiting for a free export slot... (30s)
🚀 Started export: agua_ndwi_20_tile_004_lon-64.86_lat-55.05.geojson  →  /Azolla Filiculoides - Geojson Lagunas
⏳ Waiting for a free export slot... (30s)
⏳ Waiting for a free export slot... (30s)
🚀 Started export: agua_ndwi_20_tile_005_lon-70.66_lat-53.6.geojson  →  /Azolla Filiculoides - Geojson Lagunas
🚀 Started export: agua_ndwi_20_tile_006_lon-69.21_lat-53.6.geojson  →  /Azolla Filiculoides - Geojson Lagunas
⏳ Waiting for a free export slot... (30s)
🚀 Started export: agua_ndwi_20_tile_007_lon-73.56_lat-52.15.geojson  →  /Az

# Tile size

In [ ]:
# After tiles = generate_tiles_over_geometry(...)
for idx, (x, y, g) in enumerate(tiles, 1):
    area_km2 = g.area(1).divide(1e6).getInfo()  # square meters → km²
    print(f"Tile {idx:03d} at lon {x:.2f}, lat {y:.2f}: {area_km2:,.0f} km² (after clipping to ARG)")

Tile 001 at lon -69.21, lat -55.05: 8,112 km² (after clipping to ARG)
Tile 002 at lon -67.76, lat -55.05: 8,261 km² (after clipping to ARG)
Tile 003 at lon -66.31, lat -55.05: 2,487 km² (after clipping to ARG)
Tile 004 at lon -64.86, lat -55.05: 534 km² (after clipping to ARG)
Tile 005 at lon -70.66, lat -53.60: 144 km² (after clipping to ARG)
Tile 006 at lon -69.21, lat -53.60: 2,734 km² (after clipping to ARG)
Tile 007 at lon -73.56, lat -52.15: 2,407 km² (after clipping to ARG)
Tile 008 at lon -72.11, lat -52.15: 15,379 km² (after clipping to ARG)
Tile 009 at lon -70.66, lat -52.15: 16,092 km² (after clipping to ARG)


In [17]:
ee.batch.Task.list()

[<Task WWW7ERXBDWG5JX7PMCUA2YYA EXPORT_FEATURES: agua_ndwi_20_tile_175_lon-63.41_lat-23.15 (COMPLETED)>,
 <Task 3MEDZH53WLYDLRXTPECZGNB5 EXPORT_FEATURES: agua_ndwi_20_tile_173_lon-66.31_lat-23.15 (COMPLETED)>,
 <Task VBC7HKB72UYM57IYFJC4NFGR EXPORT_FEATURES: agua_ndwi_20_tile_172_lon-67.76_lat-23.15 (COMPLETED)>,
 <Task NBKMK6E5I5W7MG62KAY74ST7 EXPORT_FEATURES: agua_ndwi_20_tile_171_lon-60.51_lat-24.6 (COMPLETED)>,
 <Task JVZHYT22UFI56K73BXCYRVWR EXPORT_FEATURES: agua_ndwi_20_tile_169_lon-63.41_lat-24.6 (COMPLETED)>,
 <Task IGJSSNPLWTICUL6CNYHPAFV4 EXPORT_FEATURES: agua_ndwi_20_tile_168_lon-64.86_lat-24.6 (COMPLETED)>,
 <Task BX2CQYC5BTK5PAAHKGWNZ52O EXPORT_FEATURES: agua_ndwi_20_tile_167_lon-66.31_lat-24.6 (COMPLETED)>,
 <Task YTZWWVDF5SCHVXQTC2CIGLDW EXPORT_FEATURES: agua_ndwi_20_tile_165_lon-69.21_lat-24.6 (COMPLETED)>,
 <Task WXJTUDSL5FACEMPFGZ4C7NQM EXPORT_FEATURES: agua_ndwi_20_tile_164_lon-54.71_lat-26.05 (COMPLETED)>,
 <Task PN5OVUNTT3EG7GEHUMHW26YU EXPORT_FEATURES: agua_ndwi_2